In [1]:
import pandas as pd
import igraph
import numpy as np
import pdb
from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
from fa2 import ForceAtlas2

#from https://github.com/idekerlab/cy-rest-R/blob/develop/workflow1_structure_based_visualization.R

# Step 1: Network Data Preparation

Load network from NDEx

In [2]:
# nice_cx_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='c060aff9-86d4-11e7-a10d-0ac135e8bacf')
nice_cx_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')
# nice_cx_network =  ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='ff8e7def-6e09-11e9-848d-0ac135e8bacf')

In [3]:
nice_cx_network.print_summary()

Name: CoCaNet2
Nodes: 36
Edges: 37
Node Attributes: 706
Edge Attributes: 517



In [4]:
nice_cx_from_server_df=nice_cx_network.to_pandas_dataframe()

In [5]:
nice_cx_from_server_df.head()

,source,interaction,target
0,druggable,pp,TS
1,DHFR,pp,MAPK1
2,CHEK1,pp,BLM
3,IMPDH1,pp,MTIF2
4,IMPDH1,pp,ING5


Convert it to simple edge list

In [6]:
edgelist=nice_cx_from_server_df.iloc[:,[0,2]]

Convert DF to undirected igraph object
This is a PPI network, so import as undirected.

In [7]:
tuples=[tuple(x) for x in edgelist.values]
g_original = igraph.Graph.TupleList(tuples,directed=False)

Extract componentes (individual connected subgraphs)

In [8]:
subgraphs=g_original.decompose()

Pick largest subgraph

In [9]:
tmp=[i.vcount() for i in subgraphs]
largeset_subgraph = subgraphs[tmp.index(max(tmp))]

Remove duplicate edges

In [10]:
g = largeset_subgraph.simplify(multiple=True, loops=True)
g.name = "Yeast network"

# Step 2: Basic statistical analysis

Global Network Statics

In [11]:
print("Density:",g.density())
print("Transitivity:",g.transitivity_undirected())

Density: 0.07881773399014778
Transitivity: 0.045454545454545456


Node statistics

In [12]:
#g_closeness = g.vs.closeness() # Closeness Centrarity
#g_degree = g.vs.degree() # Degree
#g_pagerank = g.vs.pagerank(directed = False) # PageRank
#g_vs_betweenness = g.vs.betweenness() # Betweenness Centrarity

Edge statics

In [13]:
#g_es_betweenness = g.es.edge_betweenness() # Edge Betweenness

# Step 3: Community Detection: Try multiple algorithms

Clustering

In [14]:
communities_greedy = g.community_fastgreedy().as_clustering()
communities_leading = g.community_leading_eigenvector()
communities_label_propagation = g.community_label_propagation()

In [15]:
v_community_greedy = communities_greedy.membership
v_community_leading = communities_leading.membership
v_community_label_propagation = communities_label_propagation.membership

In [16]:
def getCommunityEdge(g, community):
    num_edges = g.ecount()
    edge_community = [0]*num_edges
    comms = community
    sources=[i for i, _ in g.get_edgelist()]
    targets=[j for _, j in g.get_edgelist()]
    for i in range(num_edges):
        sidx = sources[i]
        tidx = targets[i]
        source = comms[sidx]
        target = comms[tidx]
        
        if source == target:
            edge_community[i] = source+1
    return edge_community
    

In [17]:
e_commnity_greedy = getCommunityEdge(g, v_community_greedy)
e_commnity_leading = getCommunityEdge(g, v_community_leading)
e_community_label_propagation = getCommunityEdge(g, v_community_label_propagation)

## Convert igraph to networkx

In [18]:
"""
def search_interaction(df,src,tgt):
    if len(df[(df['source']==src) & (df['target']==tgt)]):
        return df[(df['source']==src) & (df['target']==tgt)]['interaction'].values[0]
    else:
        return df[(df['source']==tgt) & (df['target']==src)]['interaction'].values[0]
"""

"\ndef search_interaction(df,src,tgt):\n    if len(df[(df['source']==src) & (df['target']==tgt)]):\n        return df[(df['source']==src) & (df['target']==tgt)]['interaction'].values[0]\n    else:\n        return df[(df['source']==tgt) & (df['target']==src)]['interaction'].values[0]\n"

In [19]:
# sources=[g.vs['name'][i] for i, _ in g.get_edgelist()]
# targets=[g.vs['name'][j] for _, j in g.get_edgelist()]
# interaction=[search_interaction(nice_cx_from_server_df,i ,j) for i,j in zip(sources,targets)]
# edgeprop=e_commnity_greedy

In [20]:
G_nx=nx.Graph()

for(i,j),k in zip(g.get_edgelist(),e_commnity_greedy):
    G_nx.add_edge(g.vs['name'][i],g.vs['name'][j],communities=k)

## Setting CyRest

In [21]:
from py2cytoscape.data.cyrest_client import CyRestClient
from py2cytoscape.data.util_network import NetworkUtil as util
from py2cytoscape.data.style import StyleUtil as s_util
import py2cytoscape

HOST = '192.168.255.96'
PORT = 1234

In [22]:
cy = CyRestClient(ip=HOST, port=PORT)

# 現在のセッションのクリア
cy.session.delete()

In [23]:
g_cy=cy.network.create_from_igraph(g)

# from py2cytoscape import cyrest

# cy2=cyrest.cyclient()
# print(type(cy2))

# cy2 = cyrest.network.get_porperties(G_nx)
# print(type(cy2))

## Layout using by ForceAtlas2

In [24]:
import networkx as nx
from fa2 import ForceAtlas2
import matplotlib.pyplot as plt

# G = nx.random_geometric_graph(400, 0.2)

forceatlas2 = ForceAtlas2(
                        # Behavior alternatives
                        outboundAttractionDistribution=True,  # Dissuade hubs
                        linLogMode=False,  # NOT IMPLEMENTED
                        adjustSizes=False,  # Prevent overlap (NOT IMPLEMENTED)
                        edgeWeightInfluence=1.0,

                        # Performance
                        jitterTolerance=1.0,  # Tolerance
                        barnesHutOptimize=True,
                        barnesHutTheta=1.2,
                        multiThreaded=False,  # NOT IMPLEMENTED

                        # Tuning
                        scalingRatio=20,
                        strongGravityMode=False,
                        gravity=0.5,

                        # Log
                        verbose=True)

positions = forceatlas2.forceatlas2_networkx_layout(G_nx, pos=None, iterations=2000)
nx.draw_networkx_nodes(G_nx, positions, node_size=20, with_labels=False, node_color="blue", alpha=0.4)
nx.draw_networkx_edges(G_nx, positions, edge_color="green", alpha=0.05)
plt.axis('off')
plt.show()


100%|██████████| 2000/2000 [00:00<00:00, 2881.52it/s]


BarnesHut Approximation  took  0.15  seconds
Repulsion forces  took  0.13  seconds
Gravitational forces  took  0.03  seconds
Attraction forces  took  0.03  seconds
AdjustSpeedAndApplyForces step  took  0.09  seconds


/usr/local/lib/python3.6/dist-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


<Figure size 640x480 with 1 Axes>

### Convert "positions" to "certesian"

In [25]:
certesian = [{'node':i,'x': float(positions[n][0]),'y': float(positions[n][1])}for i, n in enumerate(positions)]

In [26]:
ncx_from_x=ndex2.create_nice_cx_from_networkx(G_nx)

for i,cluster in enumerate(v_community_greedy):
    ncx_from_x.set_node_attribute(i,str(cluster),cluster)
    
# colors = [{"properties_of":"network","properties":{"NETWORK_BACKGROUND_PAINT":"#CCCCCC"},"properties_of":{"NODE_FILL_COLOR":{"#AAAAAA"}}}]

### Apply "certesian" to NiceCX

In [27]:
print(type(ncx_from_x.VISUAL_PROPERTIES))
# ncx_from_x.set_opaque_aspect("VISUAL_PROPERTIES",colors)

ncx_from_x.apply_style_from_network

ncx_from_x.set_opaque_aspect("cartesianLayout", certesian)

<class 'str'>


## Send to Cytoscape

In [28]:
import json
with open('py3sample_nx.cx', 'w') as outfile:
    tmp=json.dump(ncx_from_x.to_cx(), outfile)

Generating CX


In [29]:
import requests

In [30]:
url = 'http://'+HOST+':'+str(PORT)+'/v1/networks'
response1 = requests.post(url, data=tmp)
response1.text

'{"data":{},"errors":[{"status":412,"type":"urn:cytoscape:ci:cyrest-core:v1:networks:errors:7","message":"Could not parse the given network JSON: No content to map due to end-of-input\\n at [Source: org.glassfish.jersey.message.internal.EntityInputStream@60a74940; line: 1, column: 1]","link":"file:/Users/mikio_mac/CytoscapeConfiguration/3/framework-cytoscape.log"}]}'

##  upload the network

In [37]:
my_account="am"
my_password="ndexpassword"
my_server="http://public.ndexbio.org"

if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

Success.  Please continue.


In [31]:
upload_message=ncx_from_x.upload_to(my_server, my_account, my_password)
print(upload_message)

NameError: name 'my_server' is not defined

# Step 4: Send data to Cytoscape

## Community to Colors

In [ ]:
import seaborn as sns

In [ ]:
def communityToColors(members):
    basecolor='#AAAAAA'
    num_members=len(members)
    num_communities=max(members)+1
    colors = [basecolor]*num_members
    
    colorp = sns.color_palette("hls", num_communities)
    
    sns.palplot(colorp)
    
    print(num_communities,num_members)
    colorpalette=[rgb2hex(int(a[0]*255),int(a[1]*255),int(a[2]*255)) for a in colorp]
    
    for i in range(num_members):
        newcolor = colorpalette[members[i]]
        if( len(newcolor) == 0 ):
            newcolor = basecolor
        colors[i] = newcolor
    return colors

def rgb2hex(r,g,b): #RGB配列からHTMLなどに使われる16進数表現へ
    # r , g , b = 0 〜 255 # int型
    color = (r, g , b)
    html_color = '#%02X%02X%02X' % (color[0],color[1],color[2])
    return html_color


In [31]:
g.vs['color'] = communityToColors(communities_greedy.membership)
g.es['color'] = communityToColors(e_commnity_greedy)

NameError: name 'communityToColors' is not defined

In [164]:
print(g.vs['color'])

['#DB5E56', '#DB5E56', '#B8DB56', '#B8DB56', '#B8DB56', '#B8DB56', '#56DB93', '#56DB93', '#56DB93', '#5683DB', '#5683DB', '#56DB93', '#C856DB', '#C856DB', '#C856DB', '#B8DB56', '#C856DB', '#C856DB', '#56DB93', '#56DB93', '#C856DB', '#C856DB', '#C856DB', '#5683DB', '#5683DB', '#5683DB', '#DB5E56', '#DB5E56', '#DB5E56']


## BuildStyle

In [165]:
def BuildStyle(style_name, g, colors, community):
    # Prepare Defaults
    def_node_border_widh = {"visualProperty":"NODE_BORDER_WIDTH",
                           "value":0}
    def_node_transparency = {"visualProperty":"NODE_TRANSPARENCY",
                            "value":230}
    def_edge_transparency = {"visualProperty":"EDGE_TRANSPARENCY",
                            "value":120}
    def_edge_widh = {"visualProperty":"EDGE_WIDTH",
                    "value":2}
    def_network_background = {"visualProperty":"NETWORK_BACKGROUND_PAINT",
                             "value":"black"}
    
    defaults = [def_node_border_widh,
               def_node_transparency,
               def_edge_transparency,
               def_edge_widh,
               def_network_background]
    
    #Mappings
    mapping = []
    
    #Color mappings
    node_fill_color = {"mappingType":"passthrough",
                       "mappingColumns":colors,
                       "mappingColumnType":"String",
                       "visualProperty":"NODE_FILL_COLOR"}
    
    edge_color = {"mappingType":"passthrough",
                  "mappingColumn":colors,
                  "mappingColumnType":"String",
                  "visualProperty":"EDGE_STROKE_UNSELECTED_PAINT"}
    
    #Node Size Mapping
    min_betweenness = min(g.vs.betweenness())
    max_betweenness = max(g.vs.betweenness())
    
    point1 = {"value":min_betweenness,
              "lesser":"5.0",
              "equal":"5.0",
              "greater":"5.0"}
    
    point2 = {"value":max_betweenness,
              "lesser":"100.0",
              "equal":"100.0",
              "greater":"100.0"}
    
    node_size_continuous_points = [point1,point2]
    
    node_size = {"mappingType":"continuous",
                 "mappingColumn":"betweenness",
                 "mappingColumnType":"Double",
                 "visualProperty":"NODE_SIZE",
                 "points":node_size_continuous_points}
    
    edge_trans_point = {"value":1.0,
                        "lesser":"40",
                        "equal":"200",
                        "greater":"200"}
    edge_trans_continuous_points = [edge_trans_point]
    
    edge_trans = {"mappingType":"continuous",
                 "mappingColumn":community,
                 "mappingColumnType":"Double",
                 "visualProperty":"EDGE_TRANSPARENCY",
                 "points":edge_trans_continuous_points}
    
    mappings = [node_fill_color, edge_color, node_size, edge_trans]
    
    style = {"title":style_name, "default":defaults, "mapping":mappings}
    
    return style

In [166]:
style=BuildStyle('greedy',g,colors="vs['color']",community="community_fastgreedy().as_clustering().membership")

In [172]:
import json

In [186]:
a = json.dumps(style)
b = py2cytoscape.util.from_igraph(g)

In [174]:
url='http://localhost:1234/v1/styles'
response1 = requests.post(url, a)
response1.text

'{"title": "greedy_2"}'

In [170]:
def BuildStyle_simple(style_name, g, colors):
    # Prepare Defaults
    def_node_border_widh = {"visualProperty":"NODE_BORDER_WIDTH",
                           "value":0}
    def_edge_widh = {"visualProperty":"EDGE_WIDTH",
                    "value":2}
    def_network_background = {"visualProperty":"NETWORK_BACKGROUND_PAINT",
                             "value":"black"}
    
    defaults = [def_node_border_widh,
               def_network_background]
    
    #Mappings
    mapping = []
    
    #Color mappings
    node_fill_color = {"mappingType":"passthrough",
                       "mappingColumns":colors,
                       "mappingColumnType":"String",
                       "visualProperty":"NODE_FILL_COLOR"}
    
    edge_color = {"mappingType":"passthrough",
                  "mappingColumn":colors,
                  "mappingColumnType":"String",
                  "visualProperty":"EDGE_STROKE_UNSELECTED_PAINT"}
    node_label = {"mappingType":"passthrough",
                 "mappingColumn":"name",
                 "mappingColumnType":"String",
                 "visualProperty":"NODE_LABEL"}
    
    mappings = [node_fill_color, edge_color, node_label]
    
    style = {"title":style_name, "default":defaults, "mapping":mappings}
    
    return style

In [171]:
style=BuildStyle_simple('greedy',g,colors="vs['color']")